In [9]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import TypedDict, Annotated, List, Union
from langgraph.graph.message import AnyMessage, add_messages
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
import operator
from langchain_groq import ChatGroq
from langgraph.graph import END, StateGraph
from langgraph.graph import Graph
import time
load_dotenv()

True

In [10]:
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')


In [15]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192", streaming=False)

In [16]:
llm.invoke('Hola').content

'¡Hola! ¿Cómo estás?'

In [17]:
def function_1(input_1):
    complete_query = "Su tarea es proporcionar sólo el tema basado en la consulta del usuario. \
        Sólo mostrar el tema entre: [Japón , Deportes]. No incluya razonamientos. A continuación se muestra la consulta del usuario: " + input_1
    response = llm.invoke(complete_query)
    return response.content

def function_2(input_2):
    TOPIC_UPPER = input_2.upper()
    response = f"Aquí está el tema en MAYÚSCULAS: {TOPIC_UPPER}"
    return response

In [18]:
# Define a Langchain graph
workflow = Graph()

workflow.add_node("Agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('Agent', 'tool')

workflow.set_entry_point("Agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [19]:
query = "Háblame del crecimiento industrial de Japón"
app.invoke(query)

'Aquí está el tema en MAYÚSCULAS: [JAPÓN]'

In [20]:
for output in app.stream(query):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'Agent':
---
[Japón]

---

Output from node 'tool':
---
Aquí está el tema en MAYÚSCULAS: [JAPÓN]

---

